## Lab Assignment 3
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

## Business Understanding 
---
- Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs? (10 points)

## Data Understanding 
---
- Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems? (10 pts)
- Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs. (10 pts)

## Modeling and Evaluation
---
Different tasks will require different evaluation methods. Be as thorough as possible when analyzing the data you have chosen and use visualizations of the results to explain the performance and expected outcomes whenever possible. Guide the reader through your analysis with plenty of discussion of the results. Each option is broken down by:
- Train and adjust parameters (10 pts)
- Evaluate and Compare (10 pts)
- Visualize Results (10 pts)
- Summarize the Ramifications (20 pts)

Cluster Analysis
- Train: Perform cluster analysis using several clustering methods (adjust parameters).
- Eval: Use internal and/or external validation measures to describe and compare the clusterings and the clusters— how did you determine a suitable number of clusters for each method?
- Visualize: Use tables/visualization to discuss the found results. Explain each visualization in detail.
- Summarize: Describe your results. What findings are the most interesting and why?

## Deployment 
---
- Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling?  How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)?  How would your deploy your model for interested parties?  What other data should be collected?  How often would the model need to be updated, etc.? (10 pts)

`Model Useability`


Our model can be made accessbile via a simple web portal for users of all kinds. Our specific audiences would be individuals that would like to determine which olympic sport they would be most fit for based on specific attributes that they give the web application. A aspiring athlete that would like to one day join the olympics could go to our system to get a quick recommendation for the sport he or she can fit into based on body type. Unfortunately, our model takes up 2.4GB of space in a machines memory. This makes it difficult to deploy the model on most machines especially mobile apps. Any interested party would need to have a comprehensive plan designed to deploy this model at scale so as to not sacrifice accuracy. This size limitation could limit the use to a mass audience. 


`The Goal`

The goal of the model is to accurately and successfully predict the most likely olympic sport that is most fitting for the user. A user could take this information and act on it by beginning to start the process of training for the ideal sport in order to one day make the olympics. This application cannot however, gaurantee that a user will make the olympics, but it can give a user some insight on the sport that might be most suitable for the individuals success if he or she ever decided to go into the olympic games as a career option. Using this stystem is much better than shooting in the dark by selecting a sport by chance without any data to back the decision. In order for us to determine whether or not we acheived our goal we would need to study real world use cases of the system on real people. Our success would be determined by how well those people performed in their respective sports chosen by the algorithm. We would be interested in whether or not the sport chosen by the model was indeed a good fit for the user and lead to an overall successfull career in the olympic games.

`Usefulness of the model`

Our model could be most useful to future atheletes. The age range that we look at is anyone between the age of *10* and *90* (Though 90 years is not typical based on the olympic data, but there has been anomalies in this age group througout olympic history). Our interested parties would take the information from our model and hopefully develop a career based on our findings. Coaches and Personal trainors could also make use of this model as they could use it to help their clients determine what olympic sport to design their exercises around for a specific clients body type. The applications for this model are far reaching within the athletics arena.

`Deploying The Model`

Our model takes up about 2.4GB worth of memory on disk. In order to have a successful release of the model we would need to consider a cloud solution that would allow access to users that wanted consultation with the model. A good way to deploy this is to center the model around a olympic sport consultancy business. This vertical would allow future athletes to come to the consultancy and give us their athletic data. This athletic data could then be fed to the model and the model could give back a detailed report of the most optimal sport for the users success in the olympics. A part of the business could be designed to do a follow up on all the athletes that had consultations by the system. The follow up data would then be used to feed back into the system to make better and better predictions for current and future athletes. Overtime this defensible business could obtain proprietary olympic data which no one else would have. A successful deployment of this model in the real world could lead to obtaining the business of only the best athletes, and because of their success, more athletes of high caliber would consult with our model, which will in turn create a constant loop of successful olympians along with their data points that made those specific olympians successful. This could lead to a viable olympic consultancy unmatched by anyone.

`Data that should be collected`

The longer this model goes in production the more data points we could have to constantly improve the predictions. Some of the new data points we would be interested in collecting our

- number of times appeared in the olympics for those that used our model
- average age of first olympic appearance for those that used our model
- average age of first olympic podium finish for those that used our model
- Most recommended sports by our model? What is the pattern that would emerge here overtime?
- average number of time our athlete quit after getting a recommendation from our model.
- what current skills should the model consider to help optimize the model selction better. Locate factors to measure this.

These metrics would be good factors in measuring the why or why not of success around our recommendations. This feedback could be used to make our models prediction better. Ultimately we would like to be able to help all consulted atheltes win Gold in their events on a consistent basis.

`How often the model should be updated`

For the best success our model should be updated every 2 to 4 years coinciding with the new olympic data that is captured. We are fortunate enough to be in a domain that doesn't happen as often as most domains so there would not be a need for realtime updates.

## Exceptional Work 
---
- You have free reign to provide additional analyses or combine analyses. (10 pts)

Our exceptional work for this model was a production deployment to the web for a proof of concept. Our goal was to get a working olympic sport prediction system out on the web. Below is a walkthrough for what the user could expect to find when they use our application.

![Home Page](images/home_example.png)

We have spent time styling a front end user friendly application for the model which allows for an easy access point for interested parties. The user will be able to click the **Find Sport** link at the top right corner. After clicking the link they will be able to enter some information to the model for processing.

![User Form](images/find_sport_example.png)

The form is setup to take in the following...

- `Gender`: Can be either Male or Female.
- `Season`: Can be either Summer or Winter olympic games
- `Age`: The age of the athelete in question
- `Weight`: The user weight in pounds
- `Height`: The height of the user in centimeters.

These factors are what the model uses in order to make a olympic sport recommendation. Once the user enters their information they can click **Get Sport** and the system will analyze their inputs and respond with a recommendation.

![Recommendation](images/recommendation_example2.png)

Once the user enters a sport they will get an olympic sport recommendation based on their inputs. Of course, there are many things we can do with the response that the user gets. In the future we could have interesting statistics based on each sport that is recommended so that the user can get an idea of why he or she got the response they got.

You can experiment with the model here... [`Olympic Sport Recommender POC`](https://olympics-msds-7331.herokuapp.com/)